 ## 1. 데이터 수집 (Crawling)
 
 
 ### - BS(Beautiful Soup)을 활용한 데이터 수집 ( 네이버 블로그 )  
 
 
 
 
   

### 1-1. 필요한 라이브러리 설치

In [53]:
!pip install pandas   
!pip install xlsxwriter    
!pip install BS4   
!pip install wordcloud    

  Created wheel for BS4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=ccbad097c52e5a5715f7e562e70fcb5947b5f68905fca94f761c9a2bbd24340c
  Stored in directory: c:\users\yti18\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built BS4


In [1]:
import re  
import json  
import math  
import requests  
import urllib.request 
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup   
import pandas as pd
import xlsxwriter
from tqdm import tqdm  

### 1-2. 네이버 개발자 사이트에서 ID와 Secret 얻기

### 1-3. 네이버 블로그 갯수 가져오기

In [3]:
def get_blog_count(query, display): #검색어 해당하는 블로그 전체 개수 가져오기
    encode_query = urllib.parse.quote(query)
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query
    request = urllib.request.Request(search_url) 

    request.add_header("X-Naver-Client-Id", naver_client_id)  #헤더추가
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request) #실제 사이트에 해당하는 결과 요청
    response_code = response.getcode()  #반환되는 코드 확인

    if response_code is 200: # 서버가 요청을 제대로 처리한 경우
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))  # 네이버가 제공하는 JSON파일로 읽기

        print("Last build date: " + str(response_body_dict['lastBuildDate']))  # JSON 출력 중 해당 항목의 필드를 가져와 출력
        print("Total: " + str(response_body_dict['total']))
        print("Start: " + str(response_body_dict['start']))
        print("Display: " + str(response_body_dict['display']))

        if response_body_dict['total'] == 0:
            blog_count = 0
        else:
            blog_total = math.ceil(response_body_dict['total'] / int(display))

            if blog_total >= 1000:   # 검색결과를 최대 1000개까지 출력
                blog_count = 1000
            else:
                blog_count = blog_total

            print("Blog total: " + str(blog_total))
            print("Blog count: " + str(blog_count))

        return blog_count

### 1-4. 네이버 블로그 포스트 가져오기

In [3]:
def get_blog_post(query, display, start_index, sort):  # 검색된 블로그 내용 가져오기
    global no, df
    
    encode_query = urllib.parse.quote(query)  # 검색API에 요청할 변수로 query외에 display, start, sort 추가
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + \
                encode_query + "&display=" + str(display) + "&start=" + str(start_index) + "&sort=" + sort

    request = urllib.request.Request(search_url)

    request.add_header("X-Naver-Client-Id", naver_client_id)  # 아이디와 비밀번호를 헤더에 추가
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))
        for item_index in range(0, len(response_body_dict['items'])):
            try:                              # item 필드 안에 있는 title 등의 필드에 대한 내용 가져오기
                remove_html_tag = re.compile('<.*?>') #HTML 태그 제거를 위한 정규표현식 함수 사용
                title = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['title'])
                link = response_body_dict['items'][item_index]['link'].replace("amp;", "")
                description = re.sub(remove_html_tag, '', response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']

                no += 1
                post_code = requests.get(link)
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text, 'lxml')

                blog_post_content_text = ""
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com" + mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text, 'lxml')
                    
                    for blog_post_content in blog_post_soup.find_all('div', class_='se-viewer'):
                        blog_post_content_text = blog_post_content.get_text()
                        
                    for blog_post_content in blog_post_soup.find_all('div', class_='se_doc_viewer'):
                        blog_post_content_text = blog_post_content.get_text()
 
                    for blog_post_content in blog_post_soup.select('div#postViewArea'):
                        blog_post_content_text = blog_post_content.get_text()

                df.loc[no] = [title, link, description, blogger_name, blogger_link, post_date, blog_post_content_text]
                print("#", end='')
                
            except:
                item_index += 1

### 1-5. 네이버 블로그 텍스트 수집 시작
- 약 6분~8분 소요

In [5]:
# query = "SK매직 가스레인지" 부분에서 원하는 검색어만 변경.

no = 0                 # 몇개의 포스트를 저장하였는지 세기 위한 index
query = "SK매직 청소기"   # 검색을 원하는 문자열로서 UTF-8로 인코딩한다.
display = 10           # 검색 결과 출력 건수 지정, 10(기본값),100(최대)
start = 1              # 검색 시작 위치로 최대 2000까지 가능
sort = "sim"           # 정렬 옵션: sim(유사도순, 기본값), date(날짜순)

# 블로그를 DataFrame에 저장
df = pd.DataFrame(columns=("Title", "Link", "Description", "Blogger Name", "Blogger Link", "Post Date", "Post Contents"))

blog_count = get_blog_count(query, display)
for start_index in range(start, blog_count + 1, display):
    get_blog_post(query, display, start_index, sort)

NameError: name 'get_blog_count' is not defined

### 1-6. 수집된 데이터 파일로 저장한 후 불러오기

#### ** 코드파일들을 저장한 폴더 내에 data라는 이름의 하위 폴더를 미리 생성하여 향후 여러 데이터 파일들을 저장

In [1]:
# 수집된 블로그 텍스트 데이터를 분석용 csv 파일로 저장 
df.to_csv("./data/blog.csv", header=True, index=False, encoding='utf-8-sig') 


NameError: name 'df' is not defined

In [7]:
# 데이터를 읽기 위한 xslx 파일로 저장
writer = pd.ExcelWriter('./data/skm_blog.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1') # xslx 파일로 저장 
writer.close() # Pandas writer 객체 닫기

In [5]:
# 저장된 csv 파일 불러오고 확인하기

import pandas as pd
pd.set_option('display.max.colwidth', 100)
df_1 = pd.read_csv('./data/skm_blog.csv', encoding='utf-8-sig') 
df_1

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
0,물걸레 청소기 무선 추천 휴스톰-신일-SK매직...,https://blog.naver.com/autobahnkids?Redirect=Log&logNo=222543529401,물걸레 청소기 무선 추천 휴스톰-신일-SK매직-대우어플라이언스-라우쉬 [탑성비5] 벌써 날씨가 추워졌습니다. 최근엔 오랜 시간 문을 열어 환기를 시키기가 무척 까다로워졌는데...,내돈내산정보,https://blog.naver.com/autobahnkids,20211021.0,\n\n물걸레 청소기 무선 추천 휴스톰-신일-SK매직-대우어플라이언스-라우쉬 [탑성비5]벌써 날씨가 추워졌습니다. 최근엔 오랜 시간 문을 열어 환기를 시키기가 무척 까다로워...
1,"털빠짐 많은 대형견 필수품, 대용량 청소기 추천, SK매직...",https://blog.naver.com/szin93?Redirect=Log&logNo=222244834789,"영업용, 업소용 청소기라고 불리더라구요 저는 찾아보다가 SK매직 청소기로 구매했습니다 네이버에서 검색 후 129800원에 겟 요즘 일반 가정용 청소기 엄청 비싼데 이정도면 ...",vive,https://blog.naver.com/szin93,20210215.0,"\n\n\n\n\n\n\nReview\n\n\n\n\n털빠짐 많은 대형견 필수품, 대용량 청소기 추천, SK매직 공업용 청소기 \n\n\n\n\n\n섲인\n\n ・ \n20..."
2,조용하고 청소 잘되는 SK매직 듀얼스핀 무선 물걸레청소기...,https://blog.naver.com/minsoism?Redirect=Log&logNo=222205058585,"아...안돼 눈독들이지뫄 SK매직 무선 물걸레청소기 구입하고 나서도 걱정스러웠던 건 바로 소음이었어요. 먹잠먹잠하는 신생아가 있어서 소음에 민감해졌죠. 애 잘 때, 잠들려고...",민소와 내내 건강하소,https://blog.naver.com/minsoism,20210112.0,\n\n\n\n\n\n\n체험을해봤어요\n\n\n\n\n조용하고 청소 잘되는 SK매직 듀얼스핀 무선 물걸레청소기 명절선물로 추천! \n\n\n\n\n\n민소\n\n ・ \n...
3,오픈하우스 달꼬미아루의 장비포스팅 - (2) SK매직 물걸레청소기,https://blog.naver.com/aisuru0704?Redirect=Log&logNo=222147995330,뒤에 SK 매직 물걸레청소기를 사용해서 뽀송뽀송한 바닥을 관리하고 있습니다. 처음 인수하고 나서는 바닥이 너무 끈적끈적해서 입주자들의 컴플레인도 있어서 스팀청소기를 구입했어...,달꼬미아루 오픈하우스 응암,https://blog.naver.com/aisuru0704,20201118.0,\n\n\n\n\n\n\n오픈하우스 응암역점 이야기\n\n\n\n\n응암동 오픈하우스 달꼬미아루의 장비포스팅 - (2) SK매직 물걸레청소기 \n\n\n\n\n\n달꼬미아...
4,자동 물걸레 청소기 추천 SK매직 무선 듀얼스핀 물걸레...,https://blog.naver.com/jerajinbk?Redirect=Log&logNo=222225868786,SK매직 무선 듀얼스핀 물걸레 청소기 VCL-WA10 자동물걸레청소기 추천 글 / 사진 : 제라진BK #무선물걸레청소기 #물걸레청소기 #SK매직 #VCL-WA10 #듀얼스핀...,제라진BK의 제라진 블로그,https://blog.naver.com/jerajinbk,20210130.0,\n\n\n\n\n\n\nIT / 게임\n\n\n\n\n자동 물걸레 청소기 추천 SK매직 무선 듀얼스핀 물걸레 청소기 VCL-WA10 \n\n\n\n\n\n제라진BK\n\n...
...,...,...,...,...,...,...,...
555,SK매직올클린바이러스핏 살균 공기청정기라 특별해요.,https://blog.naver.com/makemage27?Redirect=Log&logNo=222294811263,아마 SK매직 올클린바이러스핏만의 독특한 기능이 아닐까 싶은데요. UV와 광촉매를 결합한 UV 아크 코어... 청소기 툴을 바꿔서 돌려도 먼지가 깔끔하게 안빠지고 면봉으로 ...,마눌꽁치's 요리 Cock!,https://blog.naver.com/makemage27,20210401.0,\n\n\n\n\n\n\n\n\n\n☞ 가전 디지털\n\n\n\n\nSK매직올클린바이러스핏 살균 공기청정기라 특별해요. \n\n\n\n\n\n마눌꽁치\n\n ・ \n202...
556,식기세척기 세제 sk매직DWA-7303D와 자연퐁스팀워시의 만남...,https://blog.naver.com/nikekkw?Redirect=Log&logNo=222282465722,로봇청소기는 남자1호 눈치보느냐고 아직 장바구니에서 잠자고 있고..(조만간 질러) 3달전쯤에 망고짱도 식세기이모님 #sk매직DWA-7303D 이오셨어요. 아주 사랑하는 이모...,똥꼬발랄한 나의 하루,https://blog.naver.com/nikekkw,20210321.0,\n\n\n\n\n\n\n\n\n\n내가 뜯어 볼께~상품리뷰\n\n\n\n\n식기세척기 세제 sk매직DWA-7303D와 자연퐁스팀워시의 만남 최강조합 실사용후기 \n\n\n...
557,[리뷰] SK매직 6인용 식기세척기(DWA1812) - 생활가전...,https://blog.naver.com/kmj290?Redirect=Log&logNo=222234279966,로봇청소기 이모들 덕분에 빨래와 청소부분은 큰 도움을 받고 있다. ▼건조기 신세계(가장 고마움... 여러 제품 중 후기가 좋았던 SK매직 6인용 식세기(DWA1812)! 때...,행복한 우리집,https://blog.naver.com/kmj290,20210206.0,\n\n\n\n\n\n\n리뷰\n\n\n\n\n[리뷰] SK매직 6인용 식기세척기(DWA1812) - 생활가전 삼대장의 완성! \n\n\n\n\n\n빈이맘\n\n ・ \n2...
558,[공유] SK매직 물세척 진공청소기를 4만원대로 구입할 수...,https://blog.naver.com/pbkim2?Redirect=Log&logNo=221391151039,청소기가 딱히 필요 없으면 모르겠지만... 이건 사야 하지 않을까요??!! 구입하러 가기 [링크] &lt;script type=&quot;text/javascript&quo...,"게임,애니,영화,그리고 잡다한 이야기",https://blog.naver.com/pbkim2,20181104.0,"\n\n 이번 십일절에 예약 구매 품목으로 나왔던데, 시중에서 7만원대가 최저가로 확인이 되는 제품이예요.그런데 이번에 특가로 49,900원이랍니다!!청소기가 딱히 필요 없..."


In [10]:
len(df_1)  # 파일이 담고 있는 전체 텍스트 개수 확인

560

In [11]:
df_1.head()  # 상위 5개 텍스트 확인

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
0,"털빠짐 많은 대형견 필수품, 대용량 청소기 추천, SK매직...",https://blog.naver.com/szin93?Redirect=Log&log...,"영업용, 업소용 청소기라고 불리더라구요 저는 찾아보다가 SK매직 청소기로 구매했습니...",vive,https://blog.naver.com/szin93,20210215.0,"\n\n\n\n\n\n\nReview\n\n\n\n\n털빠짐 많은 대형견 필수품, ..."
1,조용하고 청소 잘되는 SK매직 듀얼스핀 무선 물걸레청소기...,https://blog.naver.com/minsoism?Redirect=Log&l...,아...안돼 눈독들이지뫄 SK매직 무선 물걸레청소기 구입하고 나서도 걱정스러웠던 건...,민소와 내내 건강하소,https://blog.naver.com/minsoism,20210112.0,\n\n\n\n\n\n\n체험을해봤어요\n\n\n\n\n조용하고 청소 잘되는 SK매...
2,오픈하우스 달꼬미아루의 장비포스팅 - (2) SK매직 물걸레청소기,https://blog.naver.com/aisuru0704?Redirect=Log...,뒤에 SK 매직 물걸레청소기를 사용해서 뽀송뽀송한 바닥을 관리하고 있습니다. 처음 ...,달꼬미아루 오픈하우스 응암,https://blog.naver.com/aisuru0704,20201118.0,\n\n\n\n\n\n\n오픈하우스 응암역점 이야기\n\n\n\n\n응암동 오픈하우...
3,자동 물걸레 청소기 추천 SK매직 무선 듀얼스핀 물걸레...,https://blog.naver.com/jerajinbk?Redirect=Log&...,SK매직 무선 듀얼스핀 물걸레 청소기 VCL-WA10 자동물걸레청소기 추천 글 / ...,제라진BK의 제라진 블로그,https://blog.naver.com/jerajinbk,20210130.0,\n\n\n\n\n\n\nIT / 게임\n\n\n\n\n자동 물걸레 청소기 추천 S...
4,이마트 국민가격 물걸레 청소기 (SK매직물걸레청소기와 비교),https://blog.naver.com/onechickenleg?Redirect=...,#이마트물걸레청소기 #노브랜드물걸레청소기 #국민가격물걸레청소기 #물걸레청소기 #JJ...,REVIEWER,https://blog.naver.com/onechickenleg,20191228.0,\n\n\n\n\n\n\nSOMETHING NEW\n\n\n\n\n이마트 국민가격 ...


In [13]:
df_1.tail()   # 마지막 5개 텍스트 확인

,Title,Link,Description,Blogger Name,Blogger Link,Post Date,Post Contents
555,SK 매직 미니온 식기세척기 (SK매직 미니식기세척기),https://blog.naver.com/kumseo91?Redirect=Log&l...,SK 미니온 식기세척기 몇 일 동안 말이 많아서 저도 고민하고 있었는데 SK매직에서...,소소한 일상,https://blog.naver.com/kumseo91,20200728.0,\n\n\n\n\n\n\nreview\n\n\n\n\nSK 매직 미니온 식기세척기 ...
556,[리뷰] SK매직 6인용 식기세척기(DWA1812) - 생활가전...,https://blog.naver.com/kmj290?Redirect=Log&log...,로봇청소기 이모들 덕분에 빨래와 청소부분은 큰 도움을 받고 있다. ▼건조기 신세계(...,행복한 우리집,https://blog.naver.com/kmj290,20210206.0,\n\n\n\n\n\n\n리뷰\n\n\n\n\n[리뷰] SK매직 6인용 식기세척기(...
557,[가전] SK매직 식기세척기(무설치형) 설치 및 사용 리뷰...,https://blog.naver.com/dotorimoa?Redirect=Log&...,있는 SK매직의 DWA-2800P 모델을 구입했습니다. (사이즈: 420X435X4...,보람찬 일상 이야기★,https://blog.naver.com/dotorimoa,20210414.0,\n\n\n\n\n\n\n생활\n\n\n\n\n[가전] SK매직 식기세척기(무설치형...
558,SK매직 진공청소기 VCL-616 : 가성비 좋음,https://blog.naver.com/bb2da?Redirect=Log&logN...,다이슨 무선청소기 덕에 비슷한 무선제품이 많이 나오지만 배터리 문제도 있고 무선청소...,봉봉이와 함께 쓰는 블로그,https://blog.naver.com/bb2da,20180113.0,\n\n\n\n\n\n\n\n\n상품 리뷰\n\n\n\n\n\n\n\n\n\n\nS...
559,SK매직 올클린 공기청정기 바이러스핏 체험단을 모집해요.,https://blog.naver.com/kirasjy?Redirect=Log&lo...,SK매직 올클린 공기청정기 바이러스핏 체험단을 모집해요. 함께 이벤트에 참여해보시라...,매일이기쁨_시골 육아 이야기,https://blog.naver.com/kirasjy,20210315.0,\n\n\n\n\n\n\n리뷰.공유\n\n\n\n\nSK매직 올클린 공기청정기 바이...
